In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [32]:
def readData(filename):
    fs = open('dataset/' + filename + '.dat', 'r')    
    data = [line.strip().split('::') for line in fs]  
    fs.close()                                        
    return data
    
def trimLists(users, ratings):
    for u in users:
        del u[-1]    # remove last column: zipcode
        del u[-1]    # remove last column: occupation
    
    for r in ratings:
        del r[-1]    # remove last column: timestamp  

In [33]:
users   = readData('users')
movies  = readData('movies')
ratings = readData('ratings')
trimLists(users, ratings)

In [34]:
nrFemale = sum(1 for x in users if x[1] == 'F')
nrMale   = len(users) - nrFemale

print 'Female users: ' + `nrFemale` + '   (' + `len(users)` + ' total users)'
print 'Male   users: ' + `nrMale`   + '   (' + `len(users)` + ' total users)'

Female users: 1709   (6040 total users)
Male   users: 4331   (6040 total users)


In [35]:
udf = pd.DataFrame({ 'UserID' : pd.Series(row[0] for row in users),
                     'Gender' : pd.Series(row[1] for row in users),
                     'Age'    : pd.Series(row[2] for row in users)})

In [36]:
mdf = pd.DataFrame({ 'MovieID' : pd.Series(row[0] for row in movies),
                     'Title'   : pd.Series(row[1] for row in movies),
                     'Genres'  : pd.Series(row[2] for row in movies)})

In [37]:
rdf = pd.DataFrame({ 'UserID'  : pd.Series(row[0] for row in ratings),
                     'MovieID' : pd.Series(row[1] for row in ratings),
                     'Rating'  : pd.Series(row[2] for row in ratings)})

In [39]:
udf.head()

,Age,Gender,UserID
0,1,F,1
1,56,M,2
2,25,M,3
3,45,M,4
4,25,M,5


In [40]:
mdf.head()

,Genres,MovieID,Title
0,Animation|Children's|Comedy,1,Toy Story (1995)
1,Adventure|Children's|Fantasy,2,Jumanji (1995)
2,Comedy|Romance,3,Grumpier Old Men (1995)
3,Comedy|Drama,4,Waiting to Exhale (1995)
4,Comedy,5,Father of the Bride Part II (1995)


In [41]:
rdf.head()

,MovieID,Rating,UserID
0,1193,5,1
1,661,3,1
2,914,3,1
3,3408,4,1
4,2355,5,1


In [50]:
result = pd.merge(pd.merge(udf, rdf, on='UserID'), mdf, on='MovieID')

In [51]:
result.head()

,Age,Gender,UserID,MovieID,Rating,Genres,Title
0,1,F,1,1193,5,Drama,One Flew Over the Cuckoo's Nest (1975)
1,56,M,2,1193,5,Drama,One Flew Over the Cuckoo's Nest (1975)
2,25,M,12,1193,4,Drama,One Flew Over the Cuckoo's Nest (1975)
3,25,M,15,1193,4,Drama,One Flew Over the Cuckoo's Nest (1975)
4,50,M,17,1193,5,Drama,One Flew Over the Cuckoo's Nest (1975)


In [53]:
result.loc[result['Rating'].isin(['4','5'])]

,Age,Gender,UserID,MovieID,Rating,Genres,Title
0,1,F,1,1193,5,Drama,One Flew Over the Cuckoo's Nest (1975)
1,56,M,2,1193,5,Drama,One Flew Over the Cuckoo's Nest (1975)
2,25,M,12,1193,4,Drama,One Flew Over the Cuckoo's Nest (1975)
3,25,M,15,1193,4,Drama,One Flew Over the Cuckoo's Nest (1975)
4,50,M,17,1193,5,Drama,One Flew Over the Cuckoo's Nest (1975)
5,18,F,18,1193,4,Drama,One Flew Over the Cuckoo's Nest (1975)
6,1,M,19,1193,5,Drama,One Flew Over the Cuckoo's Nest (1975)
7,25,F,24,1193,5,Drama,One Flew Over the Cuckoo's Nest (1975)
9,45,M,33,1193,5,Drama,One Flew Over the Cuckoo's Nest (1975)
10,18,M,39,1193,5,Drama,One Flew Over the Cuckoo's Nest (1975)
